# Ananlyzing the Coffee Landscape: NYC
## Author: Visaj Nirav Shah
### September 2020

In [74]:
#importing and installing required libraries

import numpy as np 
import pandas as pd
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

import json
from pandas.io.json import json_normalize

import requests

import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn import preprocessing

!conda install -c conda-forge openpyxl --yes
import openpyxl #for exporting dataframe to Excel sheets

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim #to obtain latitude and longitude of a particular location
import folium #for generating maps

print('Required libraries were successfully imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Required libraries were successfully imported.


In [75]:
#this NYC dataset contains details about the city boroughs and their neighborhoods
#alongwith the latitude and longitude of each location

!wget -q -O 'NYData.json' https://cocl.us/new_york_dataset

print('Data was successfully downloaded!')

Data was successfully downloaded!


In [76]:
#loading data in NYData JSON in NYData

with open('NYData.json') as JSONData:
    NYData = json.load(JSONData)

In [77]:
#loading the features key

NYFeat = NYData['features']

In [78]:
#converting NYData into a Pandas Dataframe called NYNghbor

#dataframe columns
colNames = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

NYNghbor = pd.DataFrame(columns = colNames)

In [79]:
NYNghbor

,Borough,Neighborhood,Latitude,Longitude


In [80]:
#populating NYNghbor with data in NYData

for i in NYFeat:
    brgh = nghName = i['properties']['borough'] 
    nghName = i['properties']['name']
        
    nghLatLon = i['geometry']['coordinates']
    nghLat = nghLatLon[1]
    nghLon = nghLatLon[0]
    
    NYNghbor = NYNghbor.append({'Borough': brgh,
                                          'Neighborhood': nghName,
                                          'Latitude': nghLat,
                                          'Longitude': nghLon}, ignore_index=True)

In [81]:
#examining the dataframe

print("Number of boroughs: ", len(NYNghbor['Borough'].unique()))
print("Number of neighborhoods: ", NYNghbor.shape[0])
NYNghbor.set_index('Neighborhood')

Number of boroughs:  5
Number of neighborhoods:  306


,Borough,Latitude,Longitude
Neighborhood,,,
Wakefield,Bronx,40.894705,-73.847201
Co-op City,Bronx,40.874294,-73.829939
Eastchester,Bronx,40.887556,-73.827806
Fieldston,Bronx,40.895437,-73.905643
Riverdale,Bronx,40.890834,-73.912585
...,...,...,...
Hudson Yards,Manhattan,40.756658,-74.000111
Hammels,Queens,40.587338,-73.805530
Bayswater,Queens,40.611322,-73.765968


In [101]:
#creating the map

NYCity = 'New York City, NY'

NYGeoLoc = Nominatim(user_agent = "NYExplorer")
NYLoc = NYGeoLoc.geocode(NYCity)
NYLat = NYLoc.latitude
NYLong = NYLoc.longitude

NYMap = folium.Map(location = [NYLat, NYLong], zoom_start = 10)

#adding markers
for lati, long, brgh, nghbor in zip(NYNghbor['Latitude'], NYNghbor['Longitude'], NYNghbor['Borough'], NYNghbor['Neighborhood']):
    label = '{}, {}'.format(nghbor, brgh)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lati, long],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(NYMap)  
    
NYMap

In [83]:
#defining Foursquare credentials

clientID = ''
clientSecret = ''
ver = ''

In [84]:
#accessing Foursquare API and creating the resulting dataframe

def getVenues(names, latis, longs):
    
    venues = []
    for name, lati, long in zip(names, latis, longs):
            
        fsq_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}'.format(
            clientID, 
            clientSecret, 
            ver, 
            lati, 
            long
            ) #API request URL

        res = requests.get(fsq_url).json()["response"]['groups'][0]['items'] #GET request
        
        venues.append([(
            name, 
            lati, 
            long, 
            v['venue']['name'],
            v['venue']['id'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in res])

    dfVenues = pd.DataFrame([item for venue in venues for item in venue])
    dfVenues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue ID',
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(dfVenues)

In [85]:
#calling the function

NYVenues = getVenues(names = NYNghbor['Neighborhood'], 
                     latis = NYNghbor['Latitude'], 
                     longs = NYNghbor['Longitude']
                    )

In [86]:
#exporting the dataframe to Excel

NYVenues.to_excel('NYVenues.xlsx')

NYVenues = pd.read_excel('NYVenues.xlsx')

#checking our dataframe
NYVenues = NYVenues.loc[:, ~NYVenues.columns.str.contains('^Unnamed')]
NYVenues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue ID,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,4c537892fd2ea593cb077a28,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Ripe Kitchen & Bar,4d375ce799fe8eec99fd2355,40.898152,-73.838875,Caribbean Restaurant
2,Wakefield,40.894705,-73.847201,Ali's Roti Shop,4c9e50e38afca09379b2ff15,40.894036,-73.856935,Caribbean Restaurant
3,Wakefield,40.894705,-73.847201,Jackie's West Indian Bakery,4c10f6aece57c92804a682d2,40.889283,-73.843310,Caribbean Restaurant
4,Wakefield,40.894705,-73.847201,Jimbo's,4c1bed4eb306c928140763b7,40.891740,-73.858226,Burger Joint


In [87]:
#filtering coffeeshops and cafes

coffeeshop = NYVenues.loc[NYVenues['Venue Category'].str.contains('Coffee')]

cafe = NYVenues.loc[NYVenues['Venue Category'] == 'Café']

coffee = pd.concat([coffeeshop, cafe])
print("Number of locations = ", coffee.shape[0])

Number of locations =  424


In [88]:
#getting ratings of these locations from Foursquare

priceTierCol = []
ratingCol = []
likesCol = []

for idx in coffee.index:
    fsq_ratings_url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(coffee['Venue ID'][idx], 
                                                                                                          clientID, 
                                                                                                          clientSecret,
                                                                                                          ver) #API request URL
    res = requests.get(fsq_ratings_url).json()

    priceTier = res['response']['venue']['price']['tier']
    priceTierCol = priceTierCol + [priceTier]
    try:
        rating = res['response']['venue']['rating']
        ratingCol = ratingCol + [rating]
    except IndexError:
        ratingCol = ratingCol + [None]
    likes = res['response']['venue']['likes']['count']
    likesCol = likesCol + [likes]

In [89]:
#adding ratingCol to our main dataframe - coffee

coffee['Venue Price Tier'] = priceTierCol
coffee['Venue Rating'] = ratingCol
coffee['Venue Count of Likes'] = likesCol

#exporting the dataframe to Excel

coffee.to_excel('Coffee Locations.xlsx')

coffee = pd.read_excel('Coffee Locations.xlsx')

#checking our dataframe
coffee = coffee.loc[:, ~coffee.columns.str.contains('^Unnamed')]
coffee.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue ID,Venue Latitude,Venue Longitude,Venue Category,Venue Price Tier,Venue Rating,Venue Count of Likes
0,Fieldston,40.895437,-73.905643,Mon Amour Coffee & Wine,5660c06b498e4003dba169a5,40.885009,-73.900332,Coffee Shop,1,8.4,22
1,Riverdale,40.890834,-73.912585,Mon Amour Coffee & Wine,5660c06b498e4003dba169a5,40.885009,-73.900332,Coffee Shop,1,8.4,22
2,Kingsbridge,40.881687,-73.902818,Mon Amour Coffee & Wine,5660c06b498e4003dba169a5,40.885009,-73.900332,Coffee Shop,1,8.4,22
3,Kingsbridge,40.881687,-73.902818,Starbucks,55f81cd2498ee903149fcc64,40.877531,-73.905582,Coffee Shop,1,8.2,24
4,Marble Hill,40.876551,-73.910660,Starbucks,55f81cd2498ee903149fcc64,40.877531,-73.905582,Coffee Shop,1,8.2,24


In [90]:
#getting the number of neighborhoods

print("Number of neighborhoods = ", coffee.groupby('Neighborhood').count().shape[0])

Number of neighborhoods =  214


In [91]:
#we are going to divide these 214 neighborhoods into clusters

In [92]:
#first, we check if there are any None or NaN values in our dataframe

coffee.isnull().values.any()

False

In [93]:
#since the value is False, there are no None or NaN values in our dataframe

In [94]:
#now we will create a dataframe which will have 3 columns
#Neighborhood, Number of coffee locations in that neighborhood, Average rating of coffee locations in that neighborhood

NYNghborCoffee = pd.DataFrame(columns = ['Number of Coffee Locations', 'Price Tier', 'Average Rating', 'Number of Likes'])

groupByCoffee = coffee.groupby('Neighborhood') #group-by neighborhood

NYNghborCoffee['Number of Coffee Locations'] = groupByCoffee['Venue'].count()
NYNghborCoffee['Price Tier'] = groupByCoffee['Venue Price Tier'].mean()
NYNghborCoffee['Average Rating'] = groupByCoffee['Venue Rating'].mean()
NYNghborCoffee['Number of Likes'] = groupByCoffee['Venue Count of Likes'].mean()

NYNghborCoffee.head()

,Number of Coffee Locations,Price Tier,Average Rating,Number of Likes
Neighborhood,,,,
Allerton,1,1.0,8.40,12.0
Annadale,2,1.0,7.75,19.0
Arden Heights,2,1.0,7.75,19.0
Arlington,1,1.0,8.40,12.0
Arverne,1,1.0,8.70,13.0


In [ ]:
# Elbow method - we will run a loop and see which how many clusters should we create to get the best results
#the value of k for which we get a corner is the best k

sse = [] #list to maintain sum of squared distance for each k

scaler = preprocessing.StandardScaler()

for i in range(2, 11): #we will try cluster numbers from 2 to 10
    
    kmeans = KMeans(n_clusters = i, random_state = 0).fit(scaler.fit_transform(NYNghborCoffee))
    sse.append(kmeans.inertia_)

#plot

plt.xlabel('k')
plt.ylabel('SSE')
plt.plot(list(range(2, 11)), sse)

In [96]:
#as we can see, k = 5 gives the ideal result

In [97]:
scaler = preprocessing.StandardScaler()

kmeans = KMeans(n_clusters = 5, random_state = 0).fit(scaler.fit_transform(NYNghborCoffee))
kmeans.labels_

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


array([1, 2, 2, 1, 1, 4, 2, 2, 0, 1, 2, 4, 1, 0, 0, 2, 4, 2, 2, 1, 1, 1,
       2, 2, 1, 2, 1, 4, 2, 1, 1, 1, 2, 1, 0, 4, 1, 2, 4, 1, 0, 1, 1, 4,
       1, 2, 0, 4, 2, 0, 3, 2, 2, 2, 1, 0, 1, 1, 1, 2, 2, 1, 1, 0, 0, 1,
       2, 2, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 2, 0, 2, 0, 1, 0, 1, 1, 1, 2,
       1, 2, 1, 4, 2, 4, 1, 2, 2, 0, 1, 1, 1, 0, 2, 1, 3, 2, 0, 2, 0, 2,
       0, 0, 0, 2, 4, 1, 2, 2, 2, 3, 1, 4, 2, 2, 0, 0, 2, 1, 0, 1, 2, 2,
       1, 1, 2, 4, 2, 2, 0, 2, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 4, 2,
       1, 0, 1, 2, 1, 2, 1, 2, 1, 4, 1, 2, 1, 1, 1, 1, 2, 1, 1, 0, 0, 1,
       2, 1, 1, 0, 2, 1, 1, 4, 4, 0, 0, 0, 1, 1, 1, 1, 4, 1, 0, 1, 1, 3,
       1, 3, 0, 0, 0, 1, 4, 2, 2, 2, 2, 2, 1, 1, 0, 0], dtype=int32)

In [98]:
#adding the neighborhood cluster to NYNghborCoffee

clusterNum = kmeans.labels_

NYNghborCoffee['Cluster Number'] = clusterNum

NYNghborCoffee.head()

,Number of Coffee Locations,Price Tier,Average Rating,Number of Likes,Cluster Number
Neighborhood,,,,,
Allerton,1,1.0,8.40,12.0,1
Annadale,2,1.0,7.75,19.0,2
Arden Heights,2,1.0,7.75,19.0,2
Arlington,1,1.0,8.40,12.0,1
Arverne,1,1.0,8.70,13.0,1


In [99]:
#cluster characteristics

clusterChar = NYNghborCoffee.groupby('Cluster Number')
clusterChar1 = pd.DataFrame()
clusterChar1['Number of Coffee Locations'] = clusterChar['Number of Coffee Locations'].mean()
clusterChar1['Price Tier'] = clusterChar['Price Tier'].mean()
clusterChar1['Average Rating'] = clusterChar['Average Rating'].mean()
clusterChar1['Number of Likes'] = clusterChar['Number of Likes'].mean()
clusterChar1

,Number of Coffee Locations,Price Tier,Average Rating,Number of Likes
Cluster Number,,,,
0,3.731707,1.160163,8.299512,90.191870
1,1.418605,1.000000,8.514535,53.819767
2,1.587302,1.000000,7.823413,30.392857
3,2.600000,1.853333,8.633333,646.000000
4,1.894737,1.675439,8.442105,170.447368
